In [ ]:
!pip install kaggle

### Setup Kaggle Credentials:

1. Go to https://www.kaggle.com/
2. Sign in or create an account
3. Go to Account Settings (click your profile picture → Account)
4. Scroll to "API" section
5. Click "Create New API Token"
6. This downloads `kaggle.json` file
7. Upload it to this notebook or your environment

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
